# 🚀 MAF 1 Hour Handson: Travel Planner Agent

**Microsoft Agent Framework를 사용하여 여행 계획 에이전트를 처음부터 구축해보아요!**

## 📚 학습 목표

하나의 Travel Planner Agent를 점진적으로 구축하면서 다음 개념을 배웁니다:

1.  **Create Agent** - Azure OpenAI를 사용한 첫 에이전트
2.  **Multiturn Talk** - 대화 컨텍스트를 유지하는 방법
3.  **Function Tools** - 에이전트에게 Tool을 붙여보기
4.  **Human-in-the-Loop** - 중요한 작업 전 사용자 승인 받기
5.  **Memory** - 사용자 선호도를 기억하는 에이전트
6.  **Workflow** - 정해진 로직으로 액션을 순차적으로 처리하기
7.  **Observability** - 에이전트 모니터링

## ⏱️ 워크샵 타임라인 (총 60분)

| 섹션 | 내용 |
|------|------|
| Section 0 | 환경 설정, agent-framework 설치 |
| Section 1 | 기본 에이전트 생성 및 테스트 |
| Section 2 | AgentThread로 멀티턴 대화 |
| Section 3 | 항공편/호텔/날씨 Function Tools 사용|
| Section 4 | Human-in-the-Loop 예약 승인 |
| Section 5 | 메모리로 선호도 기억하기 |
| Section 6 | 워크플로우 다단계 처리 |
| Section 7 | Observability Demo |

---

# 📋 Section 0: 환경 설정 

## 필수 준비사항

### 1. Python 3.12 설치
- [Python 3.12 다운로드](https://www.python.org/downloads/) **(권장)**
- agent-framework==1.0.0b260116 사용예정이며 3.12환경이 가장 안정적입니다.

### 2. Azure CLI 설치 및 로그인
- [Azure CLI 설치](https://learn.microsoft.com/en-us/cli/azure/install-azure-cli)
- 터미널에서 `az login` 실행 → 브라우저에서 Azure 자격증명 로그인 필요!

### 3. Microsoft Foundry 프로젝트 생성 및 모델 배포
1. [Microsoft Foundry Portal에서 새로운 프로젝트 만들기](https://learn.microsoft.com/ko-kr/azure/ai-foundry/how-to/create-projects?view=foundry-classic&tabs=foundry)
2. [프로젝트에 Azure OpenAI 모델 배포 (GPT-4o 또는 GPT-4.1)](https://learn.microsoft.com/en-us/azure/ai-foundry/foundry-models/how-to/deploy-foundry-models?view=foundry-classic)
3. [사용자가 Azure OpenAI 리소스에 대해 Cognitive Services OpenAI User 또는 Contributor 역할 필요](https://learn.microsoft.com/en-us/azure/ai-foundry/openai/how-to/role-based-access-control)
4. 배포된 모델의 **엔드포인트** 및 **배포 이름** 확인 (별도로 복사해 둡니다)

---

## Step 0.1: 가상환경 생성 및 활성화

In [8]:
# 가상환경 생성 (Python 3.12)
!python -m venv .venv

⚠️ **중요**: 위 셀 실행 후, 노트북 커널을 `.venv`로 변경하세요!

- Jupyter: 우측 상단 커널 선택 → `.venv` 선택
- VS Code: 우측 상단 커널 선택 또는 `Ctrl+Shift+P` → "Select Kernel"

## Step 0.2: 가상환경 확인

In [9]:
import sys
print(f"Python 경로: {sys.executable}")
print(f"Python 버전: {sys.version}")

if '.venv' in sys.executable:
    print("\n✅ 가상환경 활성화됨!")
else:
    print("\n⚠️ 가상환경이 활성화되지 않았습니다. 커널을 .venv로 변경하세요.")

Python 경로: /Users/hangwoo_cho/Desktop/devdevdev/maf_workshop/SUPER-FAST_WORKSHOP/.venv/bin/python
Python 버전: 3.13.2 | packaged by Anaconda, Inc. | (main, Feb  6 2025, 12:55:35) [Clang 14.0.6 ]

✅ 가상환경 활성화됨!


## Step 0.3: pip 업그레이드 및 Agent Framework 설치

In [10]:
# pip 업그레이드
!pip install --upgrade pip

# Agent Framework 설치 (약 3-4분 소요)
!pip install agent-framework==1.0.0b260116

## Step 0.4: Azure CLI 인증 확인

In [11]:
from azure.identity import AzureCliCredential

try:
    credential = AzureCliCredential()
    token = credential.get_token("https://cognitiveservices.azure.com/.default")
    print("✅ Azure CLI 인증 성공!")
except Exception as e:
    print(f"❌ 인증 실패: {e}")
    print("   터미널에서 'az login'을 실행하세요.")

✅ Azure CLI 인증 성공!


## Step 0.5: Azure OpenAI 설정

아래 셀에 **본인의 엔드포인트와 배포 이름**을 입력하세요.

In [ ]:
import os

#  아래 값을 본인의 Azure OpenAI 정보로 수정하세요!
os.environ["AZURE_OPENAI_ENDPOINT"] = "https://hangw-megtq3jd-eastus2.cognitiveservices.azure.com"
os.environ["AZURE_OPENAI_CHAT_DEPLOYMENT_NAME"] = "gpt-4.1"  # 또는 gpt-4.1

print("✅ Azure OpenAI 설정 완료!")
print(f"   Endpoint: {os.environ['AZURE_OPENAI_ENDPOINT']}")
print(f"   Deployment: {os.environ['AZURE_OPENAI_CHAT_DEPLOYMENT_NAME']}")

✅ Azure OpenAI 설정 완료!
   Endpoint: https://hangw-megtq3jd-eastus2.cognitiveservices.azure.com
   Deployment: gpt-4.1


---

# Section 1: 기본 에이전트 생성

## 핵심 개념

### Agent와 ChatClient
- `AzureOpenAIChatClient`는 Azure OpenAI(또는 Foundry 기반 AOAI)와 통신하는 클라이언트입니다.
- `as_agent()`는 이 클라이언트를 “대화하고 지시를 따르는” 에이전트 형태로 감싼 래퍼입니다.

### instructions
`instructions`는 에이전트의 역할/톤/규칙을 정의하는 시스템 지침입니다. 여행 플래너의 경우 “무엇을 잘해야 하는지(추천/일정/요약)”와 “어떤 형식으로 답할지(한국어, 간결)”를 여기에 둡니다.

### run vs run_stream
- `run()`은 최종 응답을 한 번에 받습니다.
- `run_stream()`은 토큰/조각 단위로 업데이트를 받아 UI에서 실시간 출력에 유리합니다.

### stateless
에이전트는 기본적으로 상태를 저장하지 않습니다. 이전 대화를 “기억”시키려면 다음 섹션의 `AgentThread`가 필요합니다.

---

## Step 1.1: Travel Planner Agent 생성

In [13]:
from agent_framework.azure import AzureOpenAIChatClient
from azure.identity import AzureCliCredential

# Travel Planner Agent 생성
agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="""
    당신은 전문 여행 계획 어시스턴트입니다.
    - 여행지 추천
    - 액티비티 제안
    - 일정 구성 도움
    
    항상 친절하고 간결하게, 한국어로 응답합니다.
    """,
    name="TravelPlanner"
)

print("✅ Travel Planner Agent 생성 완료!")

✅ Travel Planner Agent 생성 완료!


## Step 1.2: 일반 실행 테스트

In [ ]:
result = await agent.run("봄에 유럽 주말 여행지 추천해주세요.")

print("🤖 Agent 응답:")
print(result.text)

🤖 Agent 응답:
봄에 유럽에서 주말 여행지로 매력적인 곳을 추천해드릴게요!

1. 프라하(체코)  
동화 같은 구시가지와 아름다운 야경, 그리고 맛있는 맥주까지 즐길 수 있는 도시예요. 2~3일 알차게 돌아보기 좋아요.

2. 부다페스트(헝가리)  
도나우 강을 따라 걷거나 야외 온천을 즐기기 좋고, 야경이 특히 아름답습니다.

3. 바르셀로나(스페인)  
따뜻한 봄 햇살 아래 가우디 건축물과 바르셀로네타 해변 산책을 추천해요. 다양한 타파스도 맛볼 수 있어요.

4. 암스테르담(네덜란드)  
튤립 시즌을 놓치지 마세요! 운하 투어나 자전거 투어도 즐길 수 있습니다.

5. 니스(프랑스)  
지중해의 화창한 봄 날씨와 해변, 구시가지 산책을 만끽할 수 있어요.

원하는 분위기나 관심사(예: 미식, 역사, 휴양 등)를 알려주시면 더 맞춤 추천도 도와드릴 수 있어요!


## Step 1.3: 스트리밍 실행 테스트

In [15]:
print("🤖 스트리밍 응답:")
async for update in agent.run_stream("파리에서 꼭 해야 할 3가지 활동"):
    if update.text:
        print(update.text, end="", flush=True)
print()

🤖 스트리밍 응답:
파리에서 꼭 해야 할 3가지 활동을 추천드릴게요.

1. 에펠탑 방문 및 야경 감상  
파리를 대표하는 랜드마크인 에펠탑은 직접 방문해서 꼭 올라가 보시고, 저녁에는 빛나는 야경도 감상해보세요.

2. 루브르 박물관 관람  
모나리자, 밀로의 비너스 등 세계적인 예술 작품들을 감상할 수 있는 세계 최대 규모의 미술관입니다. 미리 예약하면 더욱 편리해요!

3. 센강 유람선 타기  
노트르담 대성당, 오르세 미술관, 파리의 아름다운 다리들을 유람선을 타고 감상해보세요. 멋진 도시의 매력을 느낄 수 있습니다.

이 세 가지는 파리 첫 방문객이라면 꼭 경험해보셔야 할 최고의 액티비티입니다!


## ✅ 체크포인트 #1: 기본 에이전트

In [16]:
print("🔍 Section 1 체크")
print("=" * 50)

# 체크 1: 에이전트 생성
assert agent is not None and agent.name == "TravelPlanner"
print("✅ 에이전트 생성 성공")

# 체크 2: 응답 테스트
test = await agent.run("안녕")
assert test.text and len(test.text) > 0
print("✅ 에이전트 응답 성공")

print("\n🎉 Section 1 완료!")

🔍 Section 1 체크
✅ 에이전트 생성 성공
✅ 에이전트 응답 성공

🎉 Section 1 완료!


---

# Section 2: 멀티턴 대화

## 핵심 개념

### AgentThread로 대화 상태 유지
에이전트는 stateless이기 때문에, 멀티턴 대화에서 컨텍스트를 유지하려면 대화 상태를 담는 `AgentThread`를 만들어서 매 호출에 전달해야 합니다.

- `thread = agent.get_new_thread()` : 새로운 대화 스레드 생성
- `agent.run("...", thread=thread)` : 동일 스레드로 호출하여 대화 맥락 유지

### 멀티 스레드
하나의 에이전트 인스턴스로도 여러 개의 `AgentThread`를 만들어 독립적인 대화를 동시에 관리할 수 있습니다(각 스레드가 대화 기록을 분리 보관).

---

## Step 2.1: AgentThread 생성

In [17]:
thread = agent.get_new_thread()
print(f"✅ AgentThread 생성 완료!")

✅ AgentThread 생성 완료!


## Step 2.2: 멀티턴 대화

In [18]:
# Turn 1
r1 = await agent.run("벚꽃 시즌에 일본 여행 가고 싶어요. 언제가 좋을까요?", thread=thread)
print("🤖 Turn 1:", r1.text[:200], "...\n")

# Turn 2 - 이전 대화 기억
r2 = await agent.run("벚꽃 구경하기 좋은 도시는?", thread=thread)
print("🤖 Turn 2:", r2.text[:200], "...\n")

# Turn 3 - 전체 맥락 활용
r3 = await agent.run("첫 번째 추천 도시에서 3일 일정 짜줘", thread=thread)
print("🤖 Turn 3:", r3.text[:300], "...")

🤖 Turn 1: 벚꽃 시즌에 일본 여행을 계획하신다니 정말 멋진 선택이에요!  
일본은 지역마다 개화 시기가 약간 다릅니다.

- 도쿄, 교토, 오사카: 일반적으로 3월 말에서 4월 초가 만개 시기예요.  
- 후쿠오카 등 남쪽 지역: 3월 중순~말  
- 홋카이도(삿포로): 4월 말~5월 초

가장 인기 있는 도쿄, 교토, 오사카 기준으로 보자면 **3월 마지막 주~4월 ...

🤖 Turn 2: 벚꽃 구경에 특히 추천하는 일본의 도시는 아래와 같습니다:

1. **도쿄**  
도심 곳곳에 벚꽃 명소가 많아요. 우에노 공원, 신주쿠교엔, 메구로강 등이 유명합니다.

2. **교토**  
역사적인 사원과 함께 벚꽃을 감상할 수 있어요. 마루야마 공원, 철학의 길, 기온 지역이 명소예요.

3. **오사카**  
오사카성 공원, 카미시마 공원 등에서 화 ...

🤖 Turn 3: 네! 도쿄에서의 벚꽃 시즌 3일 여행 일정을 제안해드릴게요.

---

### 1일차: 도심 벚꽃 명소 & 시내 탐방  
- **오전**:  
  - 우에노 공원(도쿄 최대 벚꽃 명소, 산책)  
  - 우에노 동물원(관심 있으시면)  
- **점심**:  
  - 근처 이자카야 또는 돈카츠 전문점  
- **오후**:  
  - 아사쿠사 센소지(도쿄의 대표 사찰, 주변 나카미세거리 구경 및 기념품 쇼핑)  
- **저녁**:  
  - 스미다강 산책하며 벚꽃 야경 감상  
  - 아사쿠사 지역 맛집 방문

---

### 2일차: ...


## ✅ 체크포인트 #2: 멀티턴 대화

In [19]:
print("🔍 Section 2 체크")
print("=" * 50)

# 대화 기억 테스트
test_thread = agent.get_new_thread()
await agent.run("제 이름은 김철수입니다.", thread=test_thread)
name_check = await agent.run("제 이름이 뭐였죠?", thread=test_thread)

if "철수" in name_check.text or "김" in name_check.text:
    print("✅ 대화 기억 성공!")
else:
    print("⚠️ 대화 기억 확인 필요")

print("\n🎉 Section 2 완료!")

🔍 Section 2 체크
✅ 대화 기억 성공!

🎉 Section 2 완료!


---

# Section 3: Function Tools

## 핵심 개념

### Function Tool이란?
에이전트가 “필요할 때 호출할 수 있는” 사용자 정의 함수입니다. 여행 플래너에서는 항공/호텔/날씨 조회 같은 외부 데이터 조회에 씁니다.

### 어떻게 호출되나?
- 에이전트에 `tools=[...]`로 함수를 등록하면
- LLM이 사용자 요청을 보고 어떤 함수를 어떤 인자로 호출할지 결정합니다
- 프레임워크가 함수를 실행하고 결과를 다시 LLM 컨텍스트로 넣어 최종 답을 만듭니다

### 파라미터 설명의 중요성
`Annotated` + `Field(description=...)` 또는 `@ai_function`의 설명은 “함수 선택/인자 생성” 품질을 크게 좌우합니다.

---

## Step 3.1: Mock 데이터 정의

In [20]:
# Mock 데이터
MOCK_FLIGHTS = {
    "서울-도쿄": {"price": 450, "airline": "대한항공", "duration": "2시간 30분"},
    "서울-파리": {"price": 850, "airline": "에어프랑스", "duration": "12시간 15분"},
    "서울-오사카": {"price": 380, "airline": "아시아나", "duration": "2시간"},
}

MOCK_HOTELS = {
    "도쿄": [{"name": "파크 호텔", "price": 150, "rating": 4.5},
            {"name": "사쿠라 인", "price": 85, "rating": 4.2}],
    "파리": [{"name": "르 프티 호텔", "price": 120, "rating": 4.3},
            {"name": "에펠 스위트", "price": 200, "rating": 4.7}],
}

MOCK_WEATHER = {
    "도쿄": {"temp": "18°C", "condition": "맑음", "season": "봄"},
    "파리": {"temp": "15°C", "condition": "흐림", "season": "봄"},
}

print("✅ Mock 데이터 준비 완료")

✅ Mock 데이터 준비 완료


## Step 3.2: Function Tools 정의

In [21]:
from typing import Annotated
from pydantic import Field
from agent_framework import ai_function

@ai_function
def get_flight_prices(
    origin: Annotated[str, Field(description="출발 도시")],
    destination: Annotated[str, Field(description="도착 도시")]
) -> str:
    """두 도시 간 항공편 정보를 조회합니다."""
    route = f"{origin}-{destination}"
    if route in MOCK_FLIGHTS:
        f = MOCK_FLIGHTS[route]
        return f"{origin}→{destination}: ${f['price']}, {f['airline']}, {f['duration']}"
    return f"{origin}→{destination} 직항편 없음"

@ai_function
def search_hotels(
    city: Annotated[str, Field(description="도시 이름")]
) -> str:
    """도시의 호텔 목록을 검색합니다."""
    if city in MOCK_HOTELS:
        result = f"{city} 호텔:\n"
        for h in MOCK_HOTELS[city]:
            result += f"- {h['name']}: ${h['price']}/박, ⭐{h['rating']}\n"
        return result
    return f"{city} 호텔 정보 없음"

@ai_function
def get_weather(
    city: Annotated[str, Field(description="도시 이름")]
) -> str:
    """도시의 현재 날씨를 조회합니다."""
    if city in MOCK_WEATHER:
        w = MOCK_WEATHER[city]
        return f"{city}: {w['temp']}, {w['condition']} ({w['season']})"
    return f"{city} 날씨 정보 없음"

print("✅ Function Tools 정의 완료")

✅ Function Tools 정의 완료


## Step 3.3: 도구가 있는 에이전트 생성

In [23]:
agent_with_tools = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="""
    당신은 여행 계획 어시스턴트입니다.
    도구를 활용하여 정확한 정보를 제공하세요.
    - get_flight_prices: 항공편 조회
    - search_hotels: 호텔 검색
    - get_weather: 날씨 확인
    한국어로 응답합니다.
    """,
    name="TravelPlannerWithTools",
    tools=[get_flight_prices, search_hotels, get_weather]
)

print("✅ 도구가 있는 에이전트 생성 완료")

✅ 도구가 있는 에이전트 생성 완료


## Step 3.4: Tool Call 테스트

In [24]:
result = await agent_with_tools.run(
    "서울에서 도쿄로 여행 가려해요. 항공편과 호텔 정보 알려주세요."
)

print("🤖 Agent 응답 (도구 자동 호출):")
print(result.text)

🤖 Agent 응답 (도구 자동 호출):
서울 → 도쿄 항공편 정보:
- 대한항공 직항: 약 2시간 30분, 가격 $450

도쿄 호텔 추천:
- 파크 호텔: 1박 $150, 평점 4.5
- 사쿠라 인: 1박 $85, 평점 4.2

원하시는 일정이나 더 구체적인 조건(예: 숙소 등급, 위치 등)이 있으면 말씀해 주세요!


## ✅ 체크포인트 #3: Function Tools

In [25]:
print("🔍 Section 3 체크")
print("=" * 50)

# 도구 호출 테스트
test = await agent_with_tools.run("도쿄 날씨 어때?")
if "18" in test.text or "맑" in test.text or "봄" in test.text:
    print("✅ 도구 호출 성공!")
else:
    print("⚠️ 도구 호출 확인 필요")

print("\n🎉 Section 3 완료!")

🔍 Section 3 체크
✅ 도구 호출 성공!

🎉 Section 3 완료!


---

# Section 4: Human-in-the-Loop

## 핵심 개념

### 왜 필요한가?
예약/결제처럼 비용이나 위험이 있는 작업은 에이전트가 단독으로 실행하지 않고, 사용자 승인 후에만 실행하도록 만드는 패턴입니다.

### 승인 흐름
- 함수에 `approval_mode="always_require"`를 설정하면, 에이전트는 실행 대신 “승인 요청”을 반환합니다.
- 응답의 `user_input_requests`에 대기 중인 요청이 들어옵니다.
- 사용자는 승인(True) 또는 거부(False)를 결정하고, `req.create_response(...)`로 응답을 만들어 대화를 이어갑니다.

### 결과
승인 후에만 실제 함수 실행이 일어나고, 그 결과를 반영한 최종 답변이 반환됩니다.

---

## Step 4.1: 승인 필요 함수 정의

In [26]:
from agent_framework import ChatMessage, Role

@ai_function(approval_mode="always_require")
def book_flight(
    origin: Annotated[str, Field(description="출발 도시")],
    destination: Annotated[str, Field(description="도착 도시")],
    date: Annotated[str, Field(description="날짜 (YYYY-MM-DD)")]
) -> str:
    """항공편을 예약합니다. 비용 발생으로 승인 필요."""
    route = f"{origin}-{destination}"
    if route in MOCK_FLIGHTS:
        f = MOCK_FLIGHTS[route]
        return f"✅ 예약 완료! {origin}→{destination}, {date}, {f['airline']}, ${f['price']}"
    return "예약 불가"

print("✅ 승인 필요 함수 정의 완료")

✅ 승인 필요 함수 정의 완료


## Step 4.2: 예약 에이전트 생성

In [33]:
booking_agent = AzureOpenAIChatClient(credential=AzureCliCredential()).as_agent(
    instructions="여행 예약 어시스턴트입니다. 한국어로 응답합니다.",
    name="BookingAgent",
    tools=[get_flight_prices, book_flight]
)

print("✅ 예약 에이전트 생성 완료")

✅ 예약 에이전트 생성 완료


## Step 4.3: 예약 시도 및 승인 요청

In [34]:
original_query = "4월 15일 서울에서 도쿄로 항공편 예약해주세요."
result = await booking_agent.run(original_query)

if result.user_input_requests:
    print("⏸️ 승인 대기 중!\n")
    
    for req in result.user_input_requests:
        print(f"🔔 승인 요청: {req.function_call.name}")
        print(f"   인자: {req.function_call.arguments}")
    
    pending_request = result.user_input_requests[0]
else:
    print(result.text)

⏸️ 승인 대기 중!

🔔 승인 요청: book_flight
   인자: {"origin":"서울","destination":"도쿄","date":"2024-04-15"}


## Step 4.4: 승인 처리

In [35]:
# 승인!
user_approves = True
print(f"👤 사용자 결정: {'✅ 승인' if user_approves else '❌ 거부'}\n")

# create_response 메서드 사용 (True=승인, False=거부)
approval_response = pending_request.create_response(user_approves)
approval_message = ChatMessage(role=Role.USER, contents=[approval_response])

final_result = await booking_agent.run([
    original_query,
    ChatMessage(role=Role.ASSISTANT, contents=[pending_request]),
    approval_message
])

print("🤖 최종 응답:")
print(final_result.text)

👤 사용자 결정: ✅ 승인

🤖 최종 응답:
4월 15일 서울에서 도쿄로 가는 항공편이 대한항공으로 $450에 예약 완료되었습니다.  
추가로 필요한 사항이 있으신가요? (호텔, 렌터카, 다른 일정 등)


## ✅ 체크포인트 #4: Human-in-the-Loop

In [36]:
print("🔍 Section 4 체크")
print("=" * 50)

# 승인 워크플로우 테스트
test = await booking_agent.run("5월 1일 서울-도쿄 예약")
if test.user_input_requests:
    print("✅ 승인 요청 정상 작동!")
else:
    print("⚠️ 승인 요청 확인 필요")

print("\n🎉 Section 4 완료!")

🔍 Section 4 체크
✅ 승인 요청 정상 작동!

🎉 Section 4 완료!


---

# Section 5: 메모리 사용하기

## 핵심 개념

### ContextProvider
`ContextProvider`는 에이전트 호출 전/후에 로직을 끼워 넣을 수 있는 확장 포인트입니다. 대표적으로 “대화에서 정보를 추출해 저장(메모리)”하고 “다음 호출에서 그 정보를 지침으로 주입”하는 패턴에 씁니다.

### invoking / invoked
- `invoking()` : LLM 호출 전에 실행. 저장된 정보를 `Context(instructions=...)`로 주입
- `invoked()` : LLM 응답 이후 실행. 사용자 메시지에서 선호도 등을 추출해 내부 상태 업데이트

### 연결 시 주의점
- `context_provider`는 단수 파라미터입니다.
- `async with ChatAgent(...) as agent:` 패턴으로 실행해야 호출 전/후 콜백이 정상 동작합니다.

---

## Step 5.1: 메모리 클래스 정의

In [70]:
from agent_framework import ContextProvider, Context, ChatMessage
from collections.abc import Sequence
from typing import Any

class TravelPreferencesMemory(ContextProvider):
    """사용자 여행 선호도를 기억하는 메모리"""
    
    def __init__(self):
        self.preferences = {
            "budget": None,
            "style": None,
            "interests": []
        }
    
    def extract_preferences(self, text: str) -> None:
        """텍스트에서 선호도 추출 (직접 호출용)"""
        text = text.lower()
        
        # 예산
        if "저렴" in text or "가성비" in text:
            self.preferences["budget"] = "저렴/가성비"
        elif "럭셔리" in text or "고급" in text:
            self.preferences["budget"] = "럭셔리"
        
        # 스타일
        if "모험" in text or "액티비티" in text:
            self.preferences["style"] = "모험/액티비티"
        elif "휴양" in text or "휴식" in text:
            self.preferences["style"] = "휴양/휴식"
        elif "문화" in text or "역사" in text:
            self.preferences["style"] = "문화/역사"
        
        # 관심사
        for interest, keywords in [("음식", ["음식", "맛집"]), 
                                   ("쇼핑", ["쇼핑"]), 
                                   ("자연", ["자연", "산", "바다"])]:
            if any(k in text for k in keywords) and interest not in self.preferences["interests"]:
                self.preferences["interests"].append(interest)
    
    async def invoking(self, messages=None, **kwargs) -> Context | None:
        """에이전트 호출 전: 메모리 제공"""
        if any(self.preferences.values()):
            info = f"""
[사용자 선호도 정보]
- 예산: {self.preferences['budget'] or '미정'}
- 여행 스타일: {self.preferences['style'] or '미정'}
- 관심사: {', '.join(self.preferences['interests']) or '미정'}

위 정보를 참고하여 개인화된 추천을 제공하세요.
"""
            return Context(instructions=info)
        return None
    
    async def invoked(
        self,
        request_messages: ChatMessage | Sequence[ChatMessage],
        response_messages: ChatMessage | Sequence[ChatMessage] | None = None,
        invoke_exception: Exception | None = None,
        **kwargs: Any,
    ) -> None:
        """에이전트 호출 후: 선호도 추출 (자동)"""
        # request_messages를 리스트로 변환
        if isinstance(request_messages, ChatMessage):
            messages_list = [request_messages]
        else:
            messages_list = list(request_messages)
        
        # user 메시지에서 텍스트 추출
        for msg in messages_list:
            if hasattr(msg, 'role') and msg.role.value == "user":
                text = ""
                if hasattr(msg, 'contents') and msg.contents:
                    for content in msg.contents:
                        if hasattr(content, 'text'):
                            text += content.text
                        elif isinstance(content, str):
                            text += content
                        elif isinstance(content, dict) and 'text' in content:
                            text += content['text']
                
                if text:
                    self.extract_preferences(text)

print("✅ TravelPreferencesMemory 정의 완료")

✅ TravelPreferencesMemory 정의 완료


## Step 5.2: 메모리 사용 테스트

In [58]:
from agent_framework import ChatAgent

# 메모리 인스턴스 생성
memory = TravelPreferencesMemory()

# ChatClient 가져오기
chat_client = AzureOpenAIChatClient(credential=AzureCliCredential())

# 메모리가 있는 에이전트 생성 (context_provider 단수!)
agent_with_memory = ChatAgent(
    chat_client=chat_client,
    instructions="""
    당신은 여행 계획 어시스턴트입니다.
    사용자의 선호도를 기억하고 개인화된 추천을 제공합니다.
    도구를 활용하여 정확한 정보를 제공하세요.
    한국어로 응답합니다.
    """,
    tools=[get_flight_prices, search_hotels, get_weather],
    context_provider=memory  # 핵심! 단수로 전달
)

print("✅ 메모리가 있는 에이전트 생성 완료")

✅ 메모리가 있는 에이전트 생성 완료


In [59]:
# async with 패턴으로 에이전트 사용
async with agent_with_memory as agent_mem:
    thread = agent_mem.get_new_thread()
    
    # 선호도 제공 (invoked()가 자동으로 추출!)
    r1 = await agent_mem.run(
        "저는 가성비 여행을 좋아하고 문화와 역사에 관심 많아요. 음식도 좋아해요.",
        thread=thread
    )
    print("🤖 응답:", r1.text[:150], "...\n")
    
    print("🧠 메모리 상태 (자동 추출):")
    print(f"   예산: {memory.preferences['budget']}")
    print(f"   스타일: {memory.preferences['style']}")
    print(f"   관심사: {memory.preferences['interests']}")

🤖 응답: 취향을 잘 알겠습니다! 가성비 있는 여행, 문화와 역사 탐방, 그리고 맛있는 음식 경험을 중시하시는군요.

혹시 여행을 가고 싶은 특정 지역이나 도시, 또는 계절이 있으신가요? 아니면 최근에 다녀온 여행지 중 특히 좋았던 곳이나 가보고 싶은 나라가 있으신가요? 더 구체 ...

🧠 메모리 상태 (자동 추출):
   예산: 저렴/가성비
   스타일: 문화/역사
   관심사: ['음식']


In [60]:
# 메모리 기반 개인화 추천
async with agent_with_memory as agent_mem:
    thread = agent_mem.get_new_thread()
    
    # invoking()이 메모리 정보를 자동으로 주입!
    r2 = await agent_mem.run(
        "저에게 맞는 여행지 추천해주세요.",
        thread=thread
    )
    print("🤖 개인화된 추천 (invoking()이 메모리 주입):")
    print(r2.text)

🤖 개인화된 추천 (invoking()이 메모리 주입):
고객님의 선호도를 바탕으로 “저렴한 예산”, “문화/역사 여행”, “음식에 대한 관심”을 고려한 추천 여행지를 안내드리겠습니다.

1. 경주 (한국)
- 문화/역사: 신라 시대 유적(불국사, 첨성대, 대릉원 등)과 고분군이 잘 보존되어 있습니다.
- 음식: 경주교리김밥, 찰보리빵 등 특색 있는 지역 음식을 즐길 수 있습니다.
- 예산: 국내 여행지 중 비교적 저렴하며, 게스트하우스 및 저렴한 맛집이 많아 가성비가 뛰어납니다.

2. 하노이 (베트남)
- 문화/역사: 하노이 구시가지, 호안끼엠 호수, 문묘 등 베트남의 전통과 역사를 느낄 수 있습니다.
- 음식: 쌀국수(포), 분짜 등 베트남의 다양한 길거리 음식이 저렴하게 즐길 수 있습니다.
- 예산: 동남아시아 지역이라 전반적으로 숙박과 음식, 관광 비용이 저렴합니다.

3. 타이베이 (대만)
- 문화/역사: 용산사, 고궁박물관 등 유서 깊은 명소가 많습니다.
- 음식: 야시장(스린야시장, 닝샤야시장 등)에서 다양한 대만 길거리 음식과 전통 음식을 저렴하게 체험할 수 있습니다.
- 예산: 일본·유럽에 비해 합리적인 가격대로 여행 가능합니다.

이 중 특별히 관심 가는 도시가 있으신가요? 아니면 다른 국가나 도시도 추천이 필요하신가요?
선택해 주시면 숙소, 항공권, 날씨 등 더욱 정확한 정보도 안내드릴 수 있습니다!


## ✅ 체크포인트 #5: 메모리

In [61]:
print("🔍 Section 5 체크")
print("=" * 50)

if memory.preferences["budget"] and memory.preferences["style"]:
    print("✅ 메모리 추출 성공!")
    print(f"   저장된 선호도: {memory.preferences}")
else:
    print("⚠️ 메모리 추출 확인 필요")

print("\n🎉 Section 5 완료!")

🔍 Section 5 체크
✅ 메모리 추출 성공!
   저장된 선호도: {'budget': '저렴/가성비', 'style': '문화/역사', 'interests': ['음식']}

🎉 Section 5 완료!


---

# Section 6: 워크플로우

## 핵심 개념

### Workflow와 Executor
워크플로우는 여러 단계를 연결해 “순차 처리 파이프라인”을 만드는 방식입니다. 각 단계는 `@executor`로 정의하며, 입력을 받아 다음 단계로 결과를 전달합니다.

### 데이터 전달과 최종 출력
- `ctx.send_message(data)` : 다음 단계로 데이터 전달
- `ctx.yield_output(result)` : 워크플로우 최종 결과로 출력

### 타입(중요)
`WorkflowContext[...]`의 제네릭 타입은 단계 간에 주고받는 데이터 형태를 의미합니다. 단계들이 주고받는 타입이 맞지 않으면 런타임에서 의도치 않은 오류가 나기 쉽습니다.

### 연결
`WorkflowBuilder`로 단계 간 edge와 시작 노드를 설정한 뒤 `build()`로 완성합니다.

---

## Step 6.1: 워크플로우 단계 정의

In [71]:
from agent_framework import WorkflowBuilder, WorkflowContext, WorkflowOutputEvent, executor
from typing_extensions import Never

@executor(id="parse_request")
async def parse_request(text: str, ctx: WorkflowContext[dict]) -> None:
    """Step 1: 요청 파싱"""
    print("🔍 Step 1: 요청 파싱 중...")
    # 간단한 파싱 로직
    data = {"destination": "도쿄", "duration": "3일", "interests": ["문화", "음식"]}
    await ctx.send_message(data)

@executor(id="generate_itinerary")
async def generate_itinerary(data: dict, ctx: WorkflowContext[Never, str]) -> None:
    """Step 2: 일정 생성"""
    print("📝 Step 2: 일정 생성 중...")
    itinerary = f"""
🗾 {data['destination']} {data['duration']} 일정

📅 Day 1: {data['interests'][0]} 탐방
📅 Day 2: {data['interests'][1]} 투어  
📅 Day 3: 자유시간 & 출발
"""
    await ctx.yield_output(itinerary)

print("✅ 워크플로우 단계 정의 완료")

✅ 워크플로우 단계 정의 완료


## Step 6.2: 워크플로우 실행

In [72]:
# 워크플로우 빌드 (add_edge + set_start_executor)
workflow = (
    WorkflowBuilder()
    .add_edge(parse_request, generate_itinerary)  # 연결
    .set_start_executor(parse_request)             # 시작점
    .build()
)

print("🚀 Workflow 실행\n")

async for event in workflow.run_stream("도쿄 3일 문화/음식 여행"):
    print(f"📌 이벤트: {type(event).__name__}")
    if isinstance(event, WorkflowOutputEvent):
        print("\n📊 최종 결과:")
        print(event.data)

🚀 Workflow 실행

📌 이벤트: WorkflowStartedEvent
📌 이벤트: WorkflowStatusEvent
🔍 Step 1: 요청 파싱 중...
📌 이벤트: ExecutorInvokedEvent
📌 이벤트: ExecutorCompletedEvent
📌 이벤트: SuperStepStartedEvent
📝 Step 2: 일정 생성 중...
📌 이벤트: ExecutorInvokedEvent
📌 이벤트: WorkflowOutputEvent

📊 최종 결과:

🗾 도쿄 3일 일정

📅 Day 1: 문화 탐방
📅 Day 2: 음식 투어  
📅 Day 3: 자유시간 & 출발

📌 이벤트: ExecutorCompletedEvent
📌 이벤트: SuperStepCompletedEvent
📌 이벤트: WorkflowStatusEvent


## Step 6.3: 에이전트 기반 워크플로우 (Writer → Reviewer)

이번엔 **AI 에이전트**를 워크플로우 노드로 사용합니다:
- `register_agent()` - 에이전트를 워크플로우 노드로 등록
- Writer 에이전트 → Reviewer 에이전트 순차 실행

In [73]:
from agent_framework import AgentRunUpdateEvent

# ChatClient 생성 (async with 불필요)
client = AzureOpenAIChatClient(credential=AzureCliCredential())

# 에이전트 기반 워크플로우 빌드
agent_workflow = (
    WorkflowBuilder()
    # Writer 에이전트 등록
    .register_agent(
        lambda: client.as_agent(
            name="TravelWriter",
            instructions="당신은 여행 콘텐츠 작가입니다. 주어진 주제로 짧고 매력적인 여행 소개글(3-4문장)을 작성하세요. 한국어로 작성합니다."
        ),
        name="writer"
    )
    # Reviewer 에이전트 등록 (output_response=True로 최종 출력)
    .register_agent(
        lambda: client.as_agent(
            name="TravelReviewer", 
            instructions="당신은 여행 콘텐츠 편집자입니다. 작성된 글을 검토하고 간단한 피드백(2-3문장)을 제공하세요. 한국어로 피드백합니다."
        ),
        name="reviewer",
        output_response=True  # 이 에이전트의 응답이 최종 출력
    )
    .set_start_executor("writer")
    .add_edge("writer", "reviewer")
    .build()
)

print("🚀 에이전트 기반 워크플로우 실행\n")
print("=" * 50)

last_executor = None
async for event in agent_workflow.run_stream("도쿄 벚꽃 여행"):
    if isinstance(event, AgentRunUpdateEvent):
        # 에이전트가 바뀌면 줄바꿈
        if event.executor_id != last_executor:
            if last_executor:
                print("\n" + "=" * 50)
            print(f"🤖 [{event.executor_id}]:", end=" ")
            last_executor = event.executor_id
        print(event.data, end="", flush=True)
    elif isinstance(event, WorkflowOutputEvent):
        print("\n\n✅ 워크플로우 완료!")

🚀 에이전트 기반 워크플로우 실행

🤖 [TravelWriter]: 봄이 되면 도쿄는 분홍빛 벚꽃으로 물들며 환상적인 풍경을 자랑합니다. 우에노 공원이나 메구로 강을 따라 흐드러지게 핀 벚꽃 아래에서 산책을 즐겨보세요. 밤에는 벚꽃 조명이 더해져 한층 낭만적인 분위기를 만끽할 수 있습니다. 도쿄의 벚꽃 여행은 잊지 못할 봄의 추억을 선사합니다.
🤖 [TravelReviewer]: 본문이 도쿄의 벚꽃 명소와 분위기를 잘 전달하고 있습니다. 조금 더 구체적인 명소 추천이나 여행 팁이 추가되면 독자에게 더욱 실질적인 도움이 될 것 같아요.

✅ 워크플로우 완료!


## ✅ 체크포인트 #6: 워크플로우

In [74]:
print("🔍 Section 6 체크")
print("=" * 50)
print("✅ 워크플로우 실행 완료(자동체크))!")
print("\n🎉 Section 6 완료!")

🔍 Section 6 체크
✅ 워크플로우 실행 완료(자동체크))!

🎉 Section 6 완료!


---

# Section 7: 관찰성 (Observability)

## 핵심 개념

### Observability란?
에이전트 실행 중 발생하는 호출/도구 실행/지연/에러 등을 트레이스와 로그로 남겨서, 디버깅과 운영 관리를 쉽게 만드는 기능입니다. Agent Framework는 OpenTelemetry 기반으로 계측을 지원합니다.

### 빠른 활성화
```python
from agent_framework.observability import configure_otel_providers
configure_otel_providers()  # OTEL_EXPORTER_OTLP_* 환경변수를 자동으로 읽음
```

### Aspire Dashboard로 시각화(선택)
Docker로 대시보드를 띄운 뒤 OTLP 엔드포인트로 트레이스를 전송할 수 있습니다.

```bash
docker run --rm -d -p 18888:18888 -p 4317:18889 \
    mcr.microsoft.com/dotnet/aspire-dashboard:latest
```

- UI: http://localhost:18888
- OTLP: http://localhost:4317

참고: 노트북 환경에서는 콘솔 exporter 출력이 제한될 수 있어, 필요하면 별도 `.py`로 실행해서 확인합니다.

---

---

# 워크샵 완료

## 오늘 사용한 구성 요소

| 섹션 | 주제 | 핵심 API |
|------|------|----------|
| 1 | 기본 에이전트 | `as_agent()`, `instructions`, `run()`, `run_stream()` |
| 2 | 멀티턴 | `AgentThread`, `get_new_thread()` |
| 3 | 도구 호출 | `@ai_function`, `tools=[...]`, `Annotated`, `Field` |
| 4 | 승인 흐름 | `approval_mode`, `user_input_requests`, `create_response()` |
| 5 | 메모리 | `ContextProvider`, `invoking()`, `invoked()` |
| 6 | 워크플로우 | `@executor`, `WorkflowBuilder`, `ctx.send_message()`, `ctx.yield_output()` |
| 7 | 관찰성 | `configure_otel_providers()` |


- [Microsoft Agent Framework 문서](https://learn.microsoft.com/en-us/agent-framework/)
---